In [1]:
!pip install selenium
!apt-get update

     |████████████████████████████████| 968 kB 4.3 MB/s 
     |████████████████████████████████| 138 kB 63.6 MB/s 
     |████████████████████████████████| 359 kB 50.6 MB/s 
     |████████████████████████████████| 55 kB 4.5 MB/s 
     |████████████████████████████████| 4.0 MB 48.8 MB/s 
     |████████████████████████████████| 58 kB 6.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 https://developer.down

In [2]:
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver  /usr/bin
!pip install lxml

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 54 not upgraded.
Need to get 88.4 MB of archives.
After this operation, 297 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 100.0.4896.127-0ubuntu0.18.04.1 [1,142 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser a

## 엔카 크롤링 코드 (대략 완성)
- 셀레니움 사용
### 문제점
- 10만건 크롤링하려면 대략 10시간 걸릴듯 

In [6]:
# url = "http://www.encar.com/dc/dc_carsearchlist.do?carType=kor&searchType=model&TG.R=A#!%7B%22action%22%3A%22(And.Hidden.N._.CarType.Y.)%22%2C%22toggle%22%3A%7B%7D%2C%22layer%22%3A%22%22%2C%22sort%22%3A%22ModifiedDate%22%2C%22page%22%3A1%2C%22limit%22%3A20%7D"


# 셀레니움 옵션
import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
import pandas as pd
import numpy as np
import time
import datetime

options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('no-sandbox')
options.add_argument('-disable-dev-shm-usage')
options.add_argument('--start-maximized')

driver = webdriver.Chrome("chromedriver", options=options) # chromedriver가 있는 폴더 경로

# url 파싱
# driver.get(url)
# # 2초 기다려주기
# time.sleep(1)

# 만약에 팝업창 있으면 닫기
# driver.switch_to.window(driver.window_handles[1])
# driver.close()


def car_information():

  col = ['차량이름', '연식', '주행거리', '가격', '차량번호']
  car_information_df = pd.DataFrame(columns=col)

  car_name = []
  car_year = [] 
  distance_driven = []
  price = []
  # car_num = []

  # 차량이름
  for i in range(1, 22):
    name = driver.find_element_by_xpath(f'//*[@id="sr_normal"]/tr[{i}]/td[2]/a')
    car_name.append(name.text)

  # 연식
  for i in range(1, 22):
    year = driver.find_element_by_xpath(f'//*[@id="sr_normal"]/tr[{i}]/td[2]/span[1]/span[1]')
    car_year.append(year.text)

  # 주행거리
  for i in range(1, 22):
    driven = driver.find_element_by_xpath(f'//*[@id="sr_normal"]/tr[{i}]/td[2]/span[1]/span[2]')
    distance_driven.append(driven.text)

  # 가격
  for i in range(1, 22):
    pri = driver.find_element_by_xpath(f'//*[@id="sr_normal"]/tr[{i}]/td[3]')
    price.append(pri.text)

  # 차량번호
  for i in range(1, 22):
    car_n = driver.find_element_by_xpath(f'//*[@id="sr_normal"]/tr[{i}]/td[2]/span[1]/span[5]/a')
    car_n.send_keys(Keys.ENTER)
    time.sleep(2)
    driver.switch_to.window(driver.window_handles[1])
    car_n_2 = driver.find_element_by_xpath('//*[@id="bodydiv"]/div[2]/div/div[2]/table/tbody/tr[2]/td[1]')
    car_num.append(car_n_2.text)
    driver.switch_to.window(driver.window_handles[0])
  
  # 데이터 프레임에 객체 넣기
  car_information_df['차량이름'] = car_name
  car_information_df['연식'] = car_year
  car_information_df['주행거리'] = distance_driven
  car_information_df['가격'] = price
  car_information_df['차량번호'] = car_num

  # 빈 행 제거
  car_information_df['가격'].replace('', np.nan, inplace=True)
  car_information_df.dropna(subset=['가격'], inplace=True)

  return car_information_df

def all_page():

  car_information_df = pd.DataFrame()

  try:
    for i in range(1,2):
      url = f"http://www.encar.com/dc/dc_carsearchlist.do?carType=kor&searchType=model&TG.R=A#!%7B%22action%22%3A%22(And.Hidden.N._.CarType.Y.)%22%2C%22toggle%22%3A%7B%7D%2C%22layer%22%3A%22%22%2C%22sort%22%3A%22ModifiedDate%22%2C%22page%22%3A{i}%2C%22limit%22%3A20%7D"
      # url 파싱
      driver.get(url)
      # 2초 기다려주기
      time.sleep(1)
      car_information_df = car_information_df.append(car_information(), sort=False, ignore_index=True)

  except IndexError as I:
    print(I)

  finally:
    return car_information_df


def save_csv(df):
    today = str(datetime.date.today())
    outputFileName = today + 'car_information.csv'
    df.to_csv(outputFileName)
    return

# all = all_page()

# save_csv(all)

all_page()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:60: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead


""


## 연습코드

In [ ]:
# 뷰티풀솝
import requests
from bs4 import BeautifulSoup
import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
import pandas as pd
import numpy as np
import time

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('no-sandbox')
options.add_argument('-disable-dev-shm-usage')
options.add_argument('--start-maximized')

driver = webdriver.Chrome("chromedriver", options=options) # chromedriver가 있는 폴더 경로

# url 파싱

url_lst = []
for i in range(100):
  for j in range(1, 22):
    url = f'http://www.encar.com/dc/dc_cardetailview.do?pageid=dc_carsearch&listAdvType=normal&carid=32392590&view_type=hs_ad&adv_attribute=hs_ad&wtClick_korList=019&advClickPosition=kor_normal_p{i}_g{j}&tempht_arg=1HIUsGt37O29_0'
    url_lst.append(url)
print(url_lst)

for url in url_lst:
  driver.get(url)
  time.sleep(2)
  name = driver.find_element_by_xpath('//*[@id="carPic"]/div[1]/div[1]/strong')
  print(name.text)



/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


['http://www.encar.com/dc/dc_cardetailview.do?pageid=dc_carsearch&listAdvType=normal&carid=32392590&view_type=hs_ad&adv_attribute=hs_ad&wtClick_korList=019&advClickPosition=kor_normal_p0_g1&tempht_arg=1HIUsGt37O29_0', 'http://www.encar.com/dc/dc_cardetailview.do?pageid=dc_carsearch&listAdvType=normal&carid=32392590&view_type=hs_ad&adv_attribute=hs_ad&wtClick_korList=019&advClickPosition=kor_normal_p0_g2&tempht_arg=1HIUsGt37O29_0', 'http://www.encar.com/dc/dc_cardetailview.do?pageid=dc_carsearch&listAdvType=normal&carid=32392590&view_type=hs_ad&adv_attribute=hs_ad&wtClick_korList=019&advClickPosition=kor_normal_p0_g3&tempht_arg=1HIUsGt37O29_0', 'http://www.encar.com/dc/dc_cardetailview.do?pageid=dc_carsearch&listAdvType=normal&carid=32392590&view_type=hs_ad&adv_attribute=hs_ad&wtClick_korList=019&advClickPosition=kor_normal_p0_g4&tempht_arg=1HIUsGt37O29_0', 'http://www.encar.com/dc/dc_cardetailview.do?pageid=dc_carsearch&listAdvType=normal&carid=32392590&view_type=hs_ad&adv_attribute=hs

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead


NoSuchElementException: ignored

In [ ]:
# 뷰티풀솝
import requests
from bs4 import BeautifulSoup
import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
import pandas as pd
import numpy as np
import time

options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('no-sandbox')
options.add_argument('-disable-dev-shm-usage')
options.add_argument('--start-maximized')

driver = webdriver.Chrome("chromedriver", options=options) # chromedriver가 있는 폴더 경로

url = "http://www.encar.com/dc/dc_cardetailview.do?pageid=dc_carsearch&listAdvType=normal&carid=32186190&view_type=hs_ad&adv_attribute=hs_ad&wtClick_korList=019&advClickPosition=kor_normal_p1_g1&tempht_arg=1w38XTc5NiE7_0"

driver.get(url)
time.sleep(10)
name = driver.find_element_by_xpath('//*[@id="carPic"]/div[1]/div[1]/strong')
print(name.text)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead


NoSuchElementException: ignored

In [ ]:
# 뷰티풀솝
import requests
from bs4 import BeautifulSoup

url = "https://www.kbchachacha.com/public/car/detail.kbc?carSeq=22817805"

res = requests.get(url)
res.raise_for_status()

bs = BeautifulSoup(res.text,"html.parser")
print(bs)
# //*[@id="sr_normal"]/tr[1]/td[2]/a

car = bs.find_all('span', attrs={'class': 'cls'})
print(car)

for i in car:
  print(i.strong.text)



<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html lang="ko" xml:lang="ko" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="text/css" http-equiv="Content-Style-Type"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<title>국산차 검색 | 엔카 - 대한민국 No.1 중고차 플랫폼</title>
<meta content="website" property="og:type"/>
<meta content="중고차 사고팔땐 엔카, 중고차, 중고차매매사이트, 중고차시세" name="description">
<meta content="중고차매매사이트, 중고차, 중고차사이트, 중고차매매, 중고차판매" name="keywords">
<meta content="[국산차 검색 | 엔카 - 대한민국 No.1 중고차 플랫폼]" property="og:title"/>
<meta content="중고차 사고팔땐 엔카, 중고차, 중고차매매사이트, 중고차시세" property="og:description"/>
<meta content="http://www.encar.com/images/common/icon/brand_logo_400x400_v2.jpg?20526" property="og:image"/>
<meta content="http://www.encar.com/dc/dc_carsearchlist.do" 

## 연습코드 
### 셀레니움 + 뷰티불솝

In [ ]:
# 셀레니움 옵션
import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
import pandas as pd
import numpy as np
import time

options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('no-sandbox')
options.add_argument('-disable-dev-shm-usage')
options.add_argument('--start-maximized')

driver = webdriver.Chrome("chromedriver", options=options) # chromedriver가 있는 폴더 경로

# 2초 기다려주기
driver.implicitly_wait(2)

url_lst = []
for i in range(100):
  for j in range(1, 22):
    url = f'http://www.encar.com/dc/dc_cardetailview.do?pageid=dc_carsearch&listAdvType=normal&carid=32392590&view_type=hs_ad&adv_attribute=hs_ad&wtClick_korList=019&advClickPosition=kor_normal_p{i}_g{j}&tempht_arg=1HIUsGt37O29_0'
    url_lst.append(url)
print(url_lst)

for url in url_lst:
  res = requests.get(url)
  res.raise_for_status()
  bs = BeautifulSoup(res.text,"html.parser")
  # print(bs)

  car_brand = bs.find_all('div', attrs={'class': 'area_image'})
  print(car_brand)
  car_brand_1 = car_brand.find_all('strong', attrs={'class': 'prod_name'})
  car_brand_2 = car_brand_1.find_all('span', attrs={'class': 'brand'})
  # car_detail = bs.find('span', attrs={'class': 'detail'})
  print(car_brand_2.text)


  # name = driver.find_element_by_xpath('//*[@id="carPic"]/div[1]/div[1]/strong')
  # print(name.text)

['http://www.encar.com/dc/dc_cardetailview.do?pageid=dc_carsearch&listAdvType=normal&carid=32392590&view_type=hs_ad&adv_attribute=hs_ad&wtClick_korList=019&advClickPosition=kor_normal_p0_g1&tempht_arg=1HIUsGt37O29_0', 'http://www.encar.com/dc/dc_cardetailview.do?pageid=dc_carsearch&listAdvType=normal&carid=32392590&view_type=hs_ad&adv_attribute=hs_ad&wtClick_korList=019&advClickPosition=kor_normal_p0_g2&tempht_arg=1HIUsGt37O29_0', 'http://www.encar.com/dc/dc_cardetailview.do?pageid=dc_carsearch&listAdvType=normal&carid=32392590&view_type=hs_ad&adv_attribute=hs_ad&wtClick_korList=019&advClickPosition=kor_normal_p0_g3&tempht_arg=1HIUsGt37O29_0', 'http://www.encar.com/dc/dc_cardetailview.do?pageid=dc_carsearch&listAdvType=normal&carid=32392590&view_type=hs_ad&adv_attribute=hs_ad&wtClick_korList=019&advClickPosition=kor_normal_p0_g4&tempht_arg=1HIUsGt37O29_0', 'http://www.encar.com/dc/dc_cardetailview.do?pageid=dc_carsearch&listAdvType=normal&carid=32392590&view_type=hs_ad&adv_attribute=hs

AttributeError: ignored

In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://www.kbchachacha.com/public/car/detail.kbc?carSeq=22817805"

res = requests.get(url)

bs = BeautifulSoup(res.text,"html.parser")
print(bs)

text = bs.select_one('.car-buy-name').get_text()
print(text)
txt = bs.select_one('.txt-info').get_text()
print(txt)

# bs.select_one('.item_price > .s-price span').get_text()


# category = bs.find('strong', {'class': 'tit'})  
# category = category.text
# print(category)


<!DOCTYPE html>

<html lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<title>현대 더 뉴 아반떼 AD 가격, 시세 - 중고차 구매 | KB차차차</title>
<meta content="울산 지역 거래 현대 더 뉴 아반떼 AD Smartstream G1.6 (2019년식)의 가격, 연비, 주행거리, 보험사고이력을 확인하세요." name="description"/>
<meta content="KB차차차 인증중고차" property="og:title">
<meta content="(38호4514)현대더 뉴 아반떼 AD Smartstream G1.6 스타일 | 19년03월(19년식) | 70650km | 가솔린" property="og:description">
<meta content="https://m.kbchachacha.com/public/web/car/detail.kbc?carSeq=22817805" property="og:url">
<meta content="https://img.kbchachacha.com/IMG/carimg/l/img01/img2281/22817805_5758385197392869.jpg?width=720" property="og:image"/>
<link href="https://m.kbchachacha.com/public/web/car/detail.kbc?carSeq=22817805" rel="canonical"/>
<meta content="website" property="og:type"/>
<meta content="ko_KR" property="og:locale"/>
<script language="javascript">
	var IMAGE_HOST = "https://img.kbchachacha.com";
</script>
<link href="/vendor/jquery-ui

In [ ]:
from bs4 import BeautifulSoup

html = \
"""
<div class="item_price">
    <div class="o-price">
        <span><27,000원</span>
    </div>
    <div class="s-price">
        <strong><span>12,900원</span></strong>
    </div>
</div>
"""

soup = BeautifulSoup(html)

soup.select_one('.item_price').get_text()
soup.select_one('.item_price > .s-price span').get_text()

'12,900원'

In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'https://m.encar.com/pr/mprice_list.do?query=%28And.Hidden.N._.ServiceCopyCar.ORIGINAL._.%28C.CarType.Y._.%28C.Manufacturer.%C7%F6%B4%EB._.%28C.ModelGroup.%B1%D7%B7%A3%C0%FA._.Model.%B4%F5+%B4%BA+%B1%D7%B7%A3%C0%FA+IG.%29%29%29_.Mileage.range%280..200000%29.%29&idxcount=760&mnfccd=001&mdlgroupcd=004&mdlcd=162&fuelcd=&gradeheadcd=&gradecd=&year=&minMig=&maxMig=&accident=&options=&colors=&paramcount=3'
res = requests.get(url)

bs = BeautifulSoup(res.text,"html5lib")
print(bs)

text = bs.select_one('.tit_price').get_text()
print(text)
# txt = bs.select_one('.txt-info').get_text()
# print(txt)

<!DOCTYPE html>
<html class="" lang="ko"><head>
	<script>
		
			

			
		
	</script>

	<meta charset="utf-8"/>
	<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
	<meta content="index" name="WT.cg_n"/>
	<meta content="user-scalable=no, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, width=device-width" name="viewport"/>
	<meta content="telephone=no, address=no, email=no" name="format-detection"/><!-- 숫자 자동링크 방지 meta 추가 -->
	<!-- 네이버 웹마스터 매타테그 등록-->
	<meta content="828664a2ad15a94ecd1766e6271cb53caf4fae67" name="naver-site-verification"/>
    <meta content="-mlfhX62h05FI__C7tIRVHJ7LfvR-uERdJ7nlwYYsjk" name="google-site-verification"/>
	<meta content="2xnfhiu5wheuiglzcp22q1c2gjsbpf" name="facebook-domain-verification"/>


	
	
	
	
	
		<meta content="0" name="mobileoptimized"/>
		<meta content="width=device-width, height=device-height, initial-scale=1.0; user-scalable=no;" name="viewport"/>
	



	
	
	
	










<meta content="" name="_ga_userid_"/>
<meta conten

AttributeError: ignored

In [ ]:
# 셀레니움 옵션
import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
import pandas as pd
import numpy as np
import time
import datetime

options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('no-sandbox')
options.add_argument('-disable-dev-shm-usage')
options.add_argument('--start-maximized')

driver = webdriver.Chrome("chromedriver", options=options) # chromedriver가 있는 폴더 경로


url = "http://www.encar.com/dc/dc_carsearchlist.do?carType=kor&searchType=model&TG.R=A#!%7B%22action%22%3A%22(And.Hidden.N._.CarType.Y.)%22%2C%22toggle%22%3A%7B%7D%2C%22layer%22%3A%22%22%2C%22sort%22%3A%22ModifiedDate%22%2C%22page%22%3A1%2C%22limit%22%3A20%7D"

driver.get(url)
time.sleep(1)

name = driver.find_element_by_xpath('//*[@id="sr_normal"]/tr[2]/td[2]/a')
name.click()
time.sleep(10)
car_num = driver.find_element_by_class_name("list_carinfo")

print(car_num)

# for i in range(1, 22):
#   name = driver.find_element_by_xpath(f'//*[@id="sr_normal"]/tr[{i}]/td[2]/a')
#   name.click()
#   time.sleep(2)
#   car_num = driver.find_element_by_xpath('//*[@id="carPic"]/div[1]/ul[1]/li[8]')
#   print(car_num)
#   # car_name.append(name.text)
  # print(car_name)


  # 클릭
  # name.click()
  # 뒤로가기 
  # 다시 누르기 




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead


NoSuchElementException: ignored

In [ ]:
col = ['차량이름', '연식', '주행거리', '가격']
car_information_df = pd.DataFrame(columns=col)

car_name = []
car_year = [] 
distance_driven = []
price = []
car_num = []

# 차량이름
for i in range(1, 22):
  name = driver.find_element_by_xpath(f'//*[@id="sr_normal"]/tr[{i}]/td[2]/a')
  car_name.append(name.text)

# 연식
for i in range(1, 22):
  year = driver.find_element_by_xpath(f'//*[@id="sr_normal"]/tr[{i}]/td[2]/span[1]/span[1]')
  car_year.append(year.text)

# 주행거리
for i in range(1, 22):
  driven = driver.find_element_by_xpath(f'//*[@id="sr_normal"]/tr[{i}]/td[2]/span[1]/span[2]')
  distance_driven.append(driven.text)

# 가격
for i in range(1, 22):
  pri = driver.find_element_by_xpath(f'//*[@id="sr_normal"]/tr[{i}]/td[3]')
  price.append(pri.text)

# 차번호
for i in range(1, 22):
  car_n = driver.find_element_by_xpath(f'//*[@id="sr_normal"]/tr[{i}]/td[2]/span[1]/span[5]/a')
  car_n.send_keys(Keys.ENTER)
  time.sleep(2)
  driver.switch_to.window(driver.window_handles[1])
  car_n = driver.find_element_by_xpath('//*[@id="bodydiv"]/div[2]/div/div[2]/table/tbody/tr[2]/td[1]')
  car_num.append(car_n.text)
  driver.switch_to.window(driver.window_handles[0])

# 데이터 프레임에 객체 넣기
car_information_df['차량이름'] = car_name
car_information_df['연식'] = car_year
car_information_df['주행거리'] = distance_driven
car_information_df['가격'] = price
car_information_df['차량번호'] = car_num

# 빈 행 제거
car_information_df['가격'].replace('', np.nan, inplace=True)
car_information_df.dropna(subset=['가격'], inplace=True)

return car_information_df

url = "http://www.encar.com/dc/dc_carsearchlist.do?carType=kor&searchType=model&TG.R=A#!%7B%22action%22%3A%22(And.Hidden.N._.CarType.Y.)%22%2C%22toggle%22%3A%7B%7D%2C%22layer%22%3A%22%22%2C%22sort%22%3A%22ModifiedDate%22%2C%22page%22%3A1%2C%22limit%22%3A20%7D"

driver.get(url)
driver.implicitly_wait(2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: DeprecationWarning: find_element

NoSuchElementException: ignored

In [ ]:
# 셀레니움 옵션
import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
import pandas as pd
import numpy as np
import time
import datetime
import requests
from bs4 import BeautifulSoup

options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('no-sandbox')
options.add_argument('-disable-dev-shm-usage')
options.add_argument('--start-maximized')

driver = webdriver.Chrome("chromedriver", options=options)

url = "http://www.encar.com/dc/dc_carsearchlist.do?carType=kor&searchType=model&TG.R=A#!%7B%22action%22%3A%22(And.Hidden.N._.CarType.Y.)%22%2C%22toggle%22%3A%7B%7D%2C%22layer%22%3A%22%22%2C%22sort%22%3A%22ModifiedDate%22%2C%22page%22%3A1%2C%22limit%22%3A20%7D"

driver.get(url)
time.sleep(1)

car_num = []

for i in range(1, 22):
    car_n = driver.find_element_by_xpath(f'//*[@id="sr_normal"]/tr[{i}]/td[2]/a')
    car_n.send_keys(Keys.ENTER)
    driver.switch_to.window(driver.window_handles[1])
    WebDriverWait(driver, 50).until(EC.visibility_of_all_elements_located((By.XPATH,'//*[@id="carPic"]/div[1]/ul[1]/li[8]')))
    car_n_2 = driver.find_element_by_xpath('//*[@id="carPic"]/div[1]/ul[1]/li[8]')
    car_num.append(car_n_2.text)
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    
    print(car_num)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead


['43러6399']
['43러6399', '35누9126']
['43러6399', '35누9126', '81거4744']
['43러6399', '35누9126', '81거4744', '97소1893']
['43러6399', '35누9126', '81거4744', '97소1893', '46서0389']
['43러6399', '35누9126', '81거4744', '97소1893', '46서0389', '09로0534']
['43러6399', '35누9126', '81거4744', '97소1893', '46서0389', '09로0534', '33누6967']
['43러6399', '35누9126', '81거4744', '97소1893', '46서0389', '09로0534', '33누6967', '49루0540']
['43러6399', '35누9126', '81거4744', '97소1893', '46서0389', '09로0534', '33누6967', '49루0540', '67부1920']
['43러6399', '35누9126', '81거4744', '97소1893', '46서0389', '09로0534', '33누6967', '49루0540', '67부1920', '03나0688']
['43러6399', '35누9126', '81거4744', '97소1893', '46서0389', '09로0534', '33누6967', '49루0540', '67부1920', '03나0688', '07구6381']
['43러6399', '35누9126', '81거4744', '97소1893', '46서0389', '09로0534', '33누6967', '49루0540', '67부1920', '03나0688', '07구6381', '38가3384']
['43러6399', '35누9126', '81거4744', '97소1893', '46서0389', '09로0534', '33누6967', '49루0540', '67부1920', '03나0688', '07구6381', '38가3384

ElementNotInteractableException: ignored

In [ ]:
# 셀레니움 옵션
import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
import pandas as pd
import numpy as np
import time
import datetime

options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('no-sandbox')
options.add_argument('-disable-dev-shm-usage')
options.add_argument('--start-maximized')

driver = webdriver.Chrome("chromedriver", options=options)

lst = []

for i in range(6000):
    url = f"http://www.encar.com/dc/dc_carsearchlist.do?carType=kor&searchType=model&TG.R=A#!%7B%22action%22%3A%22(And.Hidden.N._.CarType.Y.)%22%2C%22toggle%22%3A%7B%7D%2C%22layer%22%3A%22%22%2C%22sort%22%3A%22ModifiedDate%22%2C%22page%22%3A{i}%2C%22limit%22%3A20%7D"
    driver.get(url)
    time.sleep(1)
    try:
      for i in range(1,22):
          tr = driver.find_elements_by_css_selector(f'#sr_normal > tr:nth-child({i}) > td.inf > a')
          for j in tr:
              href = j.get_attribute("href")
              lst.append(href)
    except:
      pass
print(lst)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
